In [45]:
import numpy as np
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.optimizers import Adam
from keras import backend as K

from rnn.data_utils import load_data, get_XY, get_test_XY

In [2]:
X, Y, m, n_X, n_Y, Tx = get_XY()
Y = Y.transpose(1, 0, 2)
print('m: ', m)
print('Tx: ', Tx)
print('n_X: ', n_X)
print('n_Y: ', n_Y)
print('shape of X:', X.shape)
print('shape of Y:', Y.shape)

m:  50
Tx:  60
n_X:  44
n_Y:  3
shape of X: (50, 60, 44)
shape of Y: (60, 50, 3)


In [3]:
# number of dimensions for the hidden state of each LSTM cell.
n_a = 64 

In [4]:
reshapor = Reshape((1, n_X))                        # Used in Step 2.B of djmodel(), below
LSTM_cell = LSTM(n_a, return_state = True)         # Used in Step 2.C
densor = Dense(n_Y)     # Used in Step 2.D

In [5]:
def djmodel(Tx, n_a, n_X, n_Y):
    """
    Implement the model
    
    Arguments:
    Tx -- length of the sequence in a corpus
    n_a -- the number of activations used in our model
    n_X -- number of features of x
    n_Y -- number of features of y
    
    Returns:
    model -- a keras instance model with n_a activations
    """
    
    # Define the input layer and specify the shape
    X = Input(shape=(Tx, n_X))
    
    # Define the initial hidden state a0 and initial cell state c0
    # using `Input`
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    ### START CODE HERE ### 
    # Step 1: Create empty list to append the outputs while you iterate (≈1 line)
    outputs = list()
    
    # Step 2: Loop
    for t in range(Tx):
        
        # Step 2.A: select the "t"th time step vector from X. 
        x = Lambda(lambda X : X[:, t, :] ) (X)
        # Step 2.B: Use reshapor to reshape x to be (1, n_X) (≈1 line)
        x = reshapor(x)
        # Step 2.C: Perform one step of the LSTM_cell
        a, _, c =  LSTM_cell(inputs=x, initial_state=[a, c])
        # Step 2.D: Apply densor to the hidden state output of LSTM_Cell
        out = densor(a)
        # Step 2.E: add the output to "outputs"
        outputs.append(out)
        
    # Step 3: Create model instance
    model = Model(inputs=[X, a0, c0], outputs=outputs)
    
    ### END CODE HERE ###
    
    return model

In [6]:
model = djmodel(Tx = Tx , n_a = n_a, n_X = n_X, n_Y = n_Y)

In [7]:
# Check your model
model.summary()

m_1[51][0]                    
                                                                 lstm_1[51][2]                    
                                                                 reshape_1[53][0]                 
                                                                 lstm_1[52][0]                    
                                                                 lstm_1[52][2]                    
                                                                 reshape_1[54][0]                 
                                                                 lstm_1[53][0]                    
                                                                 lstm_1[53][2]                    
                                                                 reshape_1[55][0]                 
                                                                 lstm_1[54][0]                    
                                                                 lstm_1[54][2]

In [8]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=opt, loss='mean_squared_error', metrics=['accuracy'])

In [9]:
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

In [10]:
model.fit([X, a0, c0], list(Y), epochs=100)

: 0.7400 - dense_1_acc_25: 0.8000 - dense_1_acc_26: 0.7400 - dense_1_acc_27: 0.7400 - dense_1_acc_28: 0.7400 - dense_1_acc_29: 0.7200 - dense_1_acc_30: 0.7600 - dense_1_acc_31: 0.7400 - dense_1_acc_32: 0.7000 - dense_1_acc_33: 0.7400 - dense_1_acc_34: 0.7400 - dense_1_acc_35: 0.8000 - dense_1_acc_36: 0.8600 - dense_1_acc_37: 0.8000 - dense_1_acc_38: 0.8400 - dense_1_acc_39: 0.7600 - dense_1_acc_40: 0.7400 - dense_1_acc_41: 0.7600 - dense_1_acc_42: 0.7600 - dense_1_acc_43: 0.8000 - dense_1_acc_44: 0.7600 - dense_1_acc_45: 0.7800 - dense_1_acc_46: 0.8400 - dense_1_acc_47: 0.8600 - dense_1_acc_48: 0.8600 - dense_1_acc_49: 0.7800 - dense_1_acc_50: 0.7400 - dense_1_acc_51: 0.7000 - dense_1_acc_52: 0.7400 - dense_1_acc_53: 0.7800 - dense_1_acc_54: 0.7200 - dense_1_acc_55: 0.7400 - dense_1_acc_56: 0.7200 - dense_1_acc_57: 0.7600 - dense_1_acc_58: 0.7800 - dense_1_acc_59: 0.7600
Epoch 89/100
50/50 [==============================] - 1s 12ms/step - loss: 5.4862 - dense_1_loss: 0.0721 - dense_1_a

In [11]:
def ball_inference_model(LSTM_cell, densor, n_X = 44, n_Y = 3, n_a = 64, Ty = 60):
    """
    Uses the trained "LSTM_cell" and "densor" from model() to generate a sequence of values.
    
    Arguments:
    LSTM_cell -- the trained "LSTM_cell" from model(), Keras layer object
    densor -- the trained "densor" from model(), Keras layer object
    n_X -- number of features of x
    n_Y -- number of features of y
    n_a -- number of units in the LSTM_cell
    Ty -- integer, number of time steps to generate
    
    Returns:
    inference_model -- Keras model instance
    """
    
    # Define the input layer and specify the shape
    X = Input(shape=(Tx, n_X))
    
    # Define the initial hidden state a0 and initial cell state c0
    # using `Input`
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    ### START CODE HERE ### 
    # Step 1: Create empty list to append the outputs while you iterate (≈1 line)
    outputs = list()
    
    # Step 2: Loop
    for t in range(Tx):
        
        # Step 2.A: select the "t"th time step vector from X. 
        x = Lambda(lambda X : X[:, t, :] ) (X)
        # Step 2.B: Use reshapor to reshape x to be (1, n_X) (≈1 line)
        x = reshapor(x)
        # Step 2.C: Perform one step of the LSTM_cell
        a, _, c =  LSTM_cell(inputs=x, initial_state=[a, c])
        # Step 2.D: Apply densor to the hidden state output of LSTM_Cell
        out = densor(a)
        # Step 2.E: add the output to "outputs"
        outputs.append(out)
        
    # Step 3: Create model instance
    model = Model(inputs=[X, a0, c0], outputs=outputs)
    
    ### END CODE HERE ###
    
    return model

In [12]:
inference_model = ball_inference_model(LSTM_cell, densor)

In [39]:
X_test, Y_test, _, _, _, _ = get_XY(dump_path='/projects/research/football/dumps/episode_done_20200105-190120854468.dump')
Y_test = Y_test.transpose(1, 0, 2)
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

print('X_shape: ', X_test.shape)
print('Y_shape: ', Y_test.shape)

X_shape:  (50, 60, 44)
Y_shape:  (60, 50, 3)


In [49]:
def predict(inference_model, X_initializer = X_test, a_initializer = a_initializer, 
                       c_initializer = c_initializer):
    """
    Predicts the next value of values using the inference model.
    
    Arguments:
    inference_model -- Keras model instance for inference time
    X_initializer -- numpy array of shape (m, Ty, n_X)
    a_initializer -- numpy array of shape (1, n_a), initializing the hidden state of the LSTM_cell
    c_initializer -- numpy array of shape (1, n_a), initializing the cell state of the LSTM_cel
    
    Returns:
    y_pres -- numpy-array of shape (Ty, n_Y), matrix of vectors representing the values generated
    """
    
    ### START CODE HERE ###
    # Step 1: Use your inference model to predict an output sequence given X_initializer, a_initializer and c_initializer.
    Y_preds = inference_model.predict([X_initializer, a_initializer, c_initializer])

    return Y_preds

In [55]:
Y_preds = predict(inference_model, X_test[0].reshape(1, X_test.shape[1], X_test.shape[2]), a_initializer, c_initializer)

print(Y_preds)

[array([[ 0.15187585, -0.00689974,  0.8667962 ]], dtype=float32), array([[0.20149781, 0.02874899, 1.1510745 ]], dtype=float32), array([[0.24600223, 0.04976754, 1.1168883 ]], dtype=float32), array([[0.2881288 , 0.05319475, 0.93645203]], dtype=float32), array([[0.3223062 , 0.04146112, 0.7542374 ]], dtype=float32), array([[0.3447703 , 0.02078757, 0.6158161 ]], dtype=float32), array([[ 0.35591882, -0.002514  ,  0.5181174 ]], dtype=float32), array([[ 0.3589748 , -0.02421459,  0.45267272]], dtype=float32), array([[ 0.35688776, -0.04216042,  0.41026115]], dtype=float32), array([[ 0.35192725, -0.05650884,  0.38275388]], dtype=float32), array([[ 0.34654537, -0.06784746,  0.36125875]], dtype=float32), array([[ 0.34443697, -0.07664467,  0.33917785]], dtype=float32), array([[ 0.3474037, -0.0824991,  0.3129217]], dtype=float32), array([[ 0.3531766 , -0.08689088,  0.2827746 ]], dtype=float32), array([[ 0.35906163, -0.08979778,  0.24980056]], dtype=float32), array([[ 0.36265674, -0.09013281,  0.21915

In [60]:
print(len(Y_preds))
print(Y_test.shape)
error = np.mean(np.square(Y_preds - Y_test[:,0,:]))
print(error)

60
(60, 50, 3)
0.16294336097136072
